In [2]:
""" 
initialisation - imports and data loading

"""
# imports

import pandas
import datetime

# load data files and set columns names

events = pandas.read_csv("/ServersExercise/Data/connectivity_events.csv")
events.columns = ['date','device_id','user_id','server_id','connection_state']

servers = pandas.read_csv("/ServersExercise/Data/servers.csv")
servers.columns = ['id','created_at','destroyed_at']


In [30]:
""" 
select first day from data

"""

# events date to datetype
events['date'] = pandas.to_datetime(events['date'])

# create column with date only
events['date_only'] = events['date'].dt.date

# get first day date beggining and end
current_day = events.loc[0]['date_only']
next_day = current_day + datetime.timedelta(days=1)

# create one day mask - set True to events that are in selected timeframe
one_day_mask = (events['date_only'] >= current_day) & (events['date_only'] < next_day)

# select first day
one_day_events = events.loc[one_day_mask]


In [39]:
# return list of events with first connection only
def get_first_connections(events_list):
    one_day_devices = events_list['device_id']
    only_first_device_connection_dict = one_day_devices.drop_duplicates()
    only_first_device_connection = pandas.DataFrame(list(only_first_device_connection_dict.items()), columns = ['original_id','device'])
    return only_first_device_connection


In [ ]:




# users dictionary initialisation
user_dic = {}

def calculate_users_devices(event):
    
    